### Iterators

In [1]:
import dis

In [2]:
dis.dis("for x in xs: do_something(name)")

  1           0 SETUP_LOOP              20 (to 22)
              2 LOAD_NAME                0 (xs)
              4 GET_ITER
        >>    6 FOR_ITER                12 (to 20)
              8 STORE_NAME               1 (x)
             10 LOAD_NAME                2 (do_something)
             12 LOAD_NAME                3 (name)
             14 CALL_FUNCTION            1
             16 POP_TOP
             18 JUMP_ABSOLUTE            6
        >>   20 POP_BLOCK
        >>   22 LOAD_CONST               0 (None)
             24 RETURN_VALUE


GET_ITER calls method `__iter__` from the argument of `for`  
FOR_ITER calls method `__next__` until the exception `StopIteration`

In [3]:
class BinaryTree:
    def __iter__(self):
        return self.inorder_iter()
    
    def preorder_iter(self):
        pass
    
    def inorder_iter(self):
        pass
    
    def postorder_iter(self):
        pass

In [5]:
a = [1, 2, 3]
next(iter(a))

1

Method `__contains__` for `in` and `not in`

In [6]:
class seq_iter:
    def __init__(self, instance):
        self.instance = instance
        self.idx = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        try:
            res = self.instance[self.idx]
        except IndexError:
            raise StopIteration
            
        self.idx += 1
        return res

### Generators

In [7]:
def g():
    print("Started")
    x = 42
    yield x
    x += 1
    yield x
    print("Done")

In [8]:
type(g)

function

In [9]:
gen = g()
type(gen)

generator

In [10]:
next(gen)

Started


42

In [11]:
next(gen)

43

In [12]:
next(gen)

Done


StopIteration: 

In [13]:
def unique(iterable, seen=None):
    seen = set(seen or [])
    for item in iterable:
        if item not in seen:
            seen.add(item)
            yield item

In [14]:
xs = [1, 1, 2, 3]
unique(xs)

<generator object unique at 0x7effd60eb150>

In [15]:
list(unique(xs))

[1, 2, 3]

In [16]:
1 in unique(xs)

True

In [17]:
def chain(*iterables):
    for iterable in iterables:
        for item in iterable:
            yield item

In [18]:
list(chain(range(3), [42]))

[0, 1, 2, 42]

In [19]:
def count(start=0):
    while True:
        yield start
        start += 1

In [20]:
next(count())

0

In [21]:
counter = count()
next(counter)

0

In [22]:
class BinaryTree:
    def __init__(self, value, left=None, right=None):
        self.value = self.value
        self.left, self.right = left, right
        
    def __iter__(self):  # inorder
        for node in self.left:
            yield node.value
        yield self.value
        for node in self.right:
            yield node.value

In [23]:
gen = (x ** 2 for x in range(10**42) if x % 2 == 1)

In [24]:
gen

<generator object <genexpr> at 0x7effd6cb6bd0>

In [25]:
next(gen)

1

In [26]:
list(filter(lambda x: x % 2 == 1, (x ** 2 for x in range(10))))

[1, 9, 25, 49, 81]

In [28]:
sum(x ** 2 for x in range(10) if x % 2 == 1)

165

`yield` as expression:

In [29]:
def g():
    res = yield
    print("Got {!r}".format(res))
    res = yield 42
    print("Got {!r}".format(res))
    

In [30]:
gen = g()

In [31]:
next(gen)

In [32]:
next(gen)

Got None


42

In [33]:
next(gen)

Got None


StopIteration: 

This is used to send the data into yield operator.

Method `send` resumes execution of the generator and sends its argument to the next `yield`.

In [34]:
gen = g()
gen.send("foobar")

TypeError: can't send non-None value to a just-started generator

To initialize the generator send `None`:

In [35]:
gen = g()
next(gen)  # gen.send(None)

In [36]:
gen = g()
gen.send(None)
gen.send("foobar")

Got 'foobar'


42

In [37]:
def g():
    try:
        yield 42
    except Exception as e:
        yield e

In [38]:
gen = g()

In [39]:
next(gen)

42

In [41]:
gen.throw(ValueError, "something is wrong")

ValueError('something is wrong')

In [42]:
gen.throw(RuntimeError, "another error")

RuntimeError: another error

In [43]:
def g():
    try:
        yield 42
    finally:
        print("Done")

In [44]:
gen = g()
next(gen)

42

In [45]:
gen.close()

Done


Coroutines:

In [46]:
def grep(pattern):
    print("Looking for {!r}".format(pattern))
    while True:
        line = yield
        if pattern in line:
            print(line)

In [47]:
gen = grep("Gotcha!")

In [48]:
next(gen)

Looking for 'Gotcha!'


In [49]:
gen.send("This line doesn't have what we're looking for")

In [50]:
gen.send("This one does. Gotcha!")

This one does. Gotcha!


Initialization decorator:

In [51]:
import functools

def coroutine(g):
    @functools.wraps(g)
    def inner(*args, **kwargs):
        gen = g(*args, **kwargs)
        next(gen)
        return gen
    return inner

In [52]:
grep = coroutine(grep)

In [53]:
gen = grep("Gotcha!")

Looking for 'Gotcha!'


In [54]:
gen.send("One more line for ya!")

`yield from`:

In [55]:
def chain(*iterables):
    for iterable in iterables:
        yield from iterable

`return` and `yield from`:

In [56]:
def f():
    yield 42
    return []

def g():
    res = yield from f()
    print("Got {!r}".format(res))

In [57]:
gen = g()
next(gen)
next(gen, None)

Got []


`contextmanager` to build context managers from generators:

In [58]:
from contextlib import contextmanager

@contextmanager
def cd(path):
    old_path = os.getcwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(old_path)

In [59]:
import tempfile
import shutil

@contextmanager
def tempdir():
    outdir = tempfile.mkdtemp()
    try:
        yield outdir
    finally:
        shutil.rmtree(outdir)
        
with tempdir() as path:
    print(path)

/tmp/tmpuawisf_r


### `itertools`

In [60]:
from itertools import islice

In [61]:
xs = range(10)

In [62]:
list(islice(xs, 3))

[0, 1, 2]

In [63]:
list(islice(xs, 3, None))

[3, 4, 5, 6, 7, 8, 9]

In [64]:
list(islice(xs, 3, 8, 2))

[3, 5, 7]

In [65]:
def take(n, iterable):
    return list(islice(iterable, n))

In [67]:
list(take(3, range(10)))

[0, 1, 2]

In [68]:
from itertools import count, cycle, repeat

In [69]:
take(3, count(0, 5))

[0, 5, 10]

In [70]:
take(3, repeat(42))

[42, 42, 42]

In [71]:
take(3, repeat(42, 2))

[42, 42]

In [72]:
from itertools import dropwhile, takewhile

In [73]:
list(dropwhile(lambda x: x < 5, range(10)))

[5, 6, 7, 8, 9]

In [74]:
it = takewhile(lambda x: x < 5, range(10))

In [75]:
it

In [76]:
list(it)

[0, 1, 2, 3, 4]

In [77]:
from itertools import chain

In [78]:
take(5, chain(range(2), range(5, 10)))

[0, 1, 5, 6, 7]

In [79]:
it = (range(x, x ** x) for x in range(2, 4))

In [80]:
take(5, chain.from_iterable(it))

[2, 3, 3, 4, 5]

In [81]:
from itertools import tee

In [82]:
it = range(3)
a, b, c = tee(it, 3)

In [83]:
list(a), list(b), list(c)

([0, 1, 2], [0, 1, 2], [0, 1, 2])

In [84]:
it = iter(range(3))  # iter - iterator, range - iterable

In [85]:
a, b = tee(it, 2)

In [86]:
used = list(it)

In [87]:
list(a), list(b)

([], [])

In [88]:
import itertools

In [89]:
list(itertools.product("AB", repeat=2))

[('A', 'A'), ('A', 'B'), ('B', 'A'), ('B', 'B')]

In [90]:
list(itertools.product("AB", repeat=3))

[('A', 'A', 'A'),
 ('A', 'A', 'B'),
 ('A', 'B', 'A'),
 ('A', 'B', 'B'),
 ('B', 'A', 'A'),
 ('B', 'A', 'B'),
 ('B', 'B', 'A'),
 ('B', 'B', 'B')]

In [91]:
list(itertools.permutations("AB"))

[('A', 'B'), ('B', 'A')]

In [92]:
list(itertools.combinations("ABC", 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [93]:
list(itertools.combinations_with_replacement("ABC", 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [94]:
def build_graph(words, mismatch_percent):
    g = ...
    n_words = len(words)
    for u, v in itertools.combinations(range(n_words), 2):
        if len(words[u]) != len(words[v]):
            continue
            
        distance = hamming(words[u], words[v])
        # ...
    return g